
# Capstone Project - Similarity between São Paulo and São José do Rio Preto

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

I currently work in São Paulo and I know and love every corner of this city, but recently I received a great job offer to work in another city, São José do Rio Preto, and now I'm moving to there.

I don't know anything about my new city and to facilitate my adaptation I would like to have a comparison between this two citys, so I can find regions similar to São Paulo in São José do Rio Preto, such as where the bars are, the historical and cultural places, the grastonomical points and etc.

So, in this project we will try to classify each district of the two cities and so identify the similar regions. In this way, will be possible to have an idea about a region of the new city based in a known region of the old city .

## Data <a name="data"></a>

The division unit chosen for classification was districts, below we have the images of the two cities broken into districts according to the official websites of their prefectures.

<img src="twoCities.jpg" width="800" />

I did an intensive search to find a dataset containing information about the districts and their coordinates, but without success. So I had to build my own dataset using google maps and adding coordinates manually. With this process, two CSV files were generated containing the districts and coordinates of each city. The files are in this repository and can be used for further analysis.

<img src="cordinates.png" width="400" />

The data of venues, like their type and location in every district can be obtained using Foursquare API.
So, for this project we will pass as parameter the locations for each district and then we will retreive all  categories of venues within a 1Km radius of that point.